In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import adata2data
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM

ModuleNotFoundError: No module named 'models'

In [4]:
#Mibitof
adata = sq.datasets.mibitof()
#feature_name=adata.obs.keys()[0] #Use for IMC dataset

#specify features to use
feature_names=['Cluster','batch']

#input of datamodule
num_features=(len(set(adata.obs[feature_names[0]])),len(set(adata.obs[feature_names[1]])))
num_genes=adata.X.shape[1]

# #IMC
# adata = sq.datasets.imc()

# #specify features to use
# feature_names=['cell type']

# #input of datamodule
# num_features=len(set(adata.obs[feature_names[0]]))
# num_genes=adata.X.shape[1]


In [5]:
dm = GraphAnnDataModule(adata=adata, feature_names=feature_names, adata2data_fn=adata2data, num_workers = 8, batch_size=1)

In [8]:
dm.setup()

In [9]:
for batch in dm.train_dataloader():
    print(batch)

[Data(x=[1045, 11], edge_index=[2, 6270], y=[1045, 36])]
[Data(x=[1023, 11], edge_index=[2, 6138], y=[1023, 36])]
[Data(x=[1241, 11], edge_index=[2, 7446], y=[1241, 36])]


In [6]:
model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)

In [7]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=100,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=100,log_every_n_steps=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 2.7 K 
1 | model_mu    | LinearSpatial   | 2.7 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)
2022-09-02 19:21:36.057206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-02 19:21:36.057268: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1023. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1241. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 1:   0%|          | 0/6 [00:00<?, ?it/s, loss=0.0385, v_num=7, val_r2_score=-1.56, val_loss=0.0374]        

/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1045. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 99: 100%|██████████| 6/6 [00:00<00:00,  7.39it/s, loss=-0.0405, v_num=7, val_r2_score=-.455, val_loss=-.0434]   
